In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

from sklearn.cluster import KMeans 
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering

In [3]:
import sys

sys.path.append('../../../src')



In [4]:
from base_exp_gen import generate_experiment as GEN_EXP
from clustering_algos import agglomerative_clustering, kmeans_clustering, box_clustering
from sklearn.model_selection import train_test_split as datasplit
from sklearn.ensemble import GradientBoostingRegressor as GDBR
from evaluations import computeATT_per_cluster, predict_cf, calculate_ite, get_homogeneity



In [5]:
np.random.seed(seed = 0)

In [6]:
pd.set_option("display.max_columns", None)


In [7]:
x_partition = [[i,i**2,2*i**4, 50000] for i in [2,4,5,10,12,20,25,33]]
def its(x):
    for i, (x1,x2,x3,x4) in enumerate(x[1:]):
        x[i+1][3]=(int(x[i][3]/(2*np.sqrt(x3/x[i][2]))))
    return x

In [8]:
a = its(x_partition)
exps = pd.DataFrame(a, columns = ['X-parts','Clusters', 'N', 'iters'])

In [9]:
name = 'adaptive-box'
exps.index.name = name

In [10]:
exps['mean-hom'] = 0
exps['std-hom'] = 0

In [11]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
box,,,,,,
0,2,4,32,50000,0,0
1,4,16,512,6250,0,0
2,5,25,1250,2000,0,0
3,10,100,20000,250,0,0
4,12,144,41472,86,0,0
5,20,400,320000,15,0,0
6,25,625,781250,4,0,0
7,33,1089,2371842,1,0,0


In [12]:
import time

In [13]:
%%time
col_select = ['x0', 'x1']
gen = []
centers = [[0.5,0.5],[0.5,0.5]]
r_small = [0, 0.325735]
r_big = [0.325735,0.46065886]
eligibilities = [2,1]

for i,row in exps.iterrows():
    c, n, iters = row.Clusters, row.N, row.iters
    homc = []
    print('running:',c,n)
    ss = time.time()
    for j in range(iters):
        ###gneerate data
        N,D = n, 2
        e = int((N)**(1/4))
        clusters = e**2
        print("Number of Clusters:{}, Number of Points:{}".format(clusters, N))
        kw_generate =  {'N':N, 'D':D,'f_gen_name': 'uniform_gen','low':0, 'high': 1}
        kw_cluster =  {'f_class_name':'circle_class4', 'centers': centers,
                       'eligibilities':eligibilities,'r_small': r_small, 'r_big':r_big}
        kw_treatment = {'f_treat_name' :'uniform_treat','choices':[0,1], 'probabilities':[0.5,0.5]}

        std = 5
        stats = np.array([[1, std], [0, std], [0, std], [1, std],[1, std],
                          [2, std]])
        kw_outcome = {'f_outcome_name': 'outcome1','treatment':'Treatment', 'cls':'C', 'stats':stats}

        data = GEN_EXP(kw_generate, kw_cluster, kw_treatment, kw_outcome).dat
        
        ### cluster data
        ### calc hom
        
        hom = calc_hom_adaptive()
        homc.append(hom)
        
    ###get stats
    ee = time.time() - ss
    print('time:',ee)
    homcnp = np.array(homc)
    gen.append(homcnp)
    mean = np.mean(homcnp)
    std = np.std(homcnp)
    exps.loc[i,'mean-hom'] = mean
    exps.loc[i,'std-hom'] = std
    
    

running: 4 32
time: 7772.299128770828
running: 16 512
time: 1816.9583611488342
running: 25 1250
time: 790.0982978343964
running: 100 20000
time: 308.4517066478729
running: 144 41472
time: 148.02998614311218
running: 400 320000
time: 78.26035165786743
running: 625 781250
time: 35.72227334976196
running: 1089 2371842
time: 20.80250859260559
CPU times: user 1h 20min 13s, sys: 17.7 s, total: 1h 20min 30s
Wall time: 3h 2min 50s


In [14]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
box,,,,,,
0,2,4,32,50000,0.671417,0.061389
1,4,16,512,6250,0.669695,0.015040
2,5,25,1250,2000,0.776799,0.010427
3,10,100,20000,250,0.937516,0.001523
4,12,144,41472,86,0.918619,0.001281
5,20,400,320000,15,0.962720,0.000296
6,25,625,781250,4,0.955020,0.000335
7,33,1089,2371842,1,0.973252,0.000000


In [139]:
def adaptive_hom(x, e, r1, r2):
    N = len(x)
    check = 9
    d = np.zeros(shape = [N, check]).astype(np.float64)
    
    xe = np.array([-e/2,0,e/2]).astype(np.float64)
    
    count = 0
    for i in range(len(xe)):
        for j in range(len(xe)):
            #if i==0 and j== 0:
                #continue
        
            d[:,count] = (x[:,0]+xe[i]-0.5)**2+(x[:,1]+xe[j]-0.5)**2
            count += 1
            
    reg1 = r2[0]**2
    reg2 = r2[1]**2
    print(reg1, reg2)

    
    d1 = np.where( d <= reg1, -1, d)
   # print(d)
    d2 = np.where( ((d1 >= reg1) & (d1 <= reg2)), -2, d1)
   # print(d)
    d3 = np.where( d2 >= reg2 , -3, d2)
    #print(d)
    
    mn = np.min(d3, axis = 1)
    mx = np.max(d3, axis = 1)
    
    return np.mean(mn == mx), [d, d1, d2, d3]

    
    

In [151]:
N,D = 2*10**6, 2
x = int((N)**(1/4))
clusters = x**2
print("Number of Clusters:{}, Number of Points:{}".format(clusters, N))
kw_generate =  {'N':N, 'D':D,'f_gen_name': 'uniform_gen','low':0, 'high': 1}
centers = [[0.5,0.5],[0.5,0.5]]
r_small = [0, 0.325735]
r_big = [0.325735,0.46065886]
eligibilities = [2,1]
kw_cluster =  {'f_class_name':'circle_class4', 'centers': centers,
               'eligibilities':eligibilities,'r_small': r_small, 'r_big':r_big}
kw_treatment = {'f_treat_name' :'uniform_treat','choices':[0,1], 'probabilities':[0.5,0.5]}

std = 5
stats = np.array([[1, std], [0, std], [0, std], [1, std],[1, std],
                  [2, std]])
kw_outcome = {'f_outcome_name': 'outcome1','treatment':'Treatment', 'cls':'C', 'stats':stats}

data = GEN_EXP(kw_generate, kw_cluster, kw_treatment, kw_outcome).dat
data = calculate_ite(data.copy(), treatment = 'Treatment',
                     counterfactual = 'Ycf', outcome = 'Y', ite_name = 'ITE')

Number of Clusters:1369, Number of Points:2000000


In [152]:
col_select = ['x0', 'x1']
values = data[col_select].values

In [153]:
res, ds = adaptive_hom(values, 1/x, np.array(r_small), np.array(r_big))

0.106103290225 0.2122065852964996


In [154]:
res, 1/x

(0.8298535, 0.02702702702702703)

In [148]:
values

array([[7.26535012e-01, 4.47099391e-01],
       [7.91187580e-01, 1.82341289e-01],
       [6.14138618e-02, 4.70300365e-01],
       ...,
       [2.20327505e-02, 1.41948813e-01],
       [7.06996249e-01, 9.70210595e-01],
       [2.82764753e-04, 9.59946366e-01]])

In [149]:
res

0.9045371

In [150]:
values

array([[7.26535012e-01, 4.47099391e-01],
       [7.91187580e-01, 1.82341289e-01],
       [6.14138618e-02, 4.70300365e-01],
       ...,
       [2.20327505e-02, 1.41948813e-01],
       [7.06996249e-01, 9.70210595e-01],
       [2.82764753e-04, 9.59946366e-01]])

In [128]:
print(ds[0].round(1))

[[0.3 0.2 0.2 ... 0.2 0.2 0.2]
 [0.1 0.1 0.1 ... 0.  0.  0.1]
 [0.1 0.1 0.1 ... 0.2 0.1 0.1]
 ...
 [0.2 0.1 0.1 ... 0.1 0.1 0.1]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.1 0.1 0.1 ... 0.1 0.1 0.1]]


In [129]:
ds[-1]


array([[-3., -3., -3., ..., -3., -3., -2.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-2., -2., -1., ..., -2., -2., -2.],
       ...,
       [-2., -2., -2., ..., -2., -2., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-2., -1., -1., ..., -2., -2., -2.]])

In [80]:
res

0.6

In [63]:
ds[0]

array([[0.16721111, 0.33000076, 0.06902495, ..., 0.13362843, 0.24086252,
        0.40365217],
       [0.16004959, 0.2948417 , 0.18815979, ..., 0.3510621 , 0.43029865,
        0.56509076],
       [0.56223518, 0.83576437, 0.3264672 , ..., 0.36422842, 0.58220206,
        0.85573125],
       ...,
       [0.66370082, 0.9628755 , 0.45725403, ..., 0.54998193, 0.79360105,
        1.09277573],
       [0.28010162, 0.48418407, 0.15310229, ..., 0.2301854 , 0.37871229,
        0.58279474],
       [0.15171958, 0.30255767, 0.04281567, ..., 0.08474984, 0.18003238,
        0.33087047]])

In [64]:
from pathlib import Path



In [16]:
p = './homogstd5/'
Path('./homogstd5/').mkdir(parents = True, exist_ok = True)
exps.to_csv(p+name+'.csv')
